In [4]:
import os

#os.environ["KERAS_BACKEND"] = "torch"  # @param ["tensorflow", "jax", "torch"]

import keras
from keras import layers
#from keras import ops

import numpy as np
import matplotlib.pyplot as plt

In [5]:
import os
import torch
import h5py
import torch.nn as nn
from tensorflow.keras.layers import Input, Conv3D, BatchNormalization, Activation, GlobalAveragePooling3D, Dense
from tensorflow.keras.models import Model
import tensorflow as tf
import torch.optim as optim
import numpy as np

from torch.utils.data import Dataset, DataLoader
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, models
def load_data(file_path, dataset_name, start_idx=0, end_idx=None):
    with h5py.File(file_path, 'r') as h5_file:
        dataset = np.array(h5_file[dataset_name][start_idx:end_idx])
    return dataset
x_train_file = '/Users/lucabernecker/Desktop/N128_local/steno_32_xxl_seg/x_train.h5'
y_train_file = '/Users/lucabernecker/Desktop/N128_local/steno_32_xxl_seg/y_train.h5'
x_dataset_name = 'x_train'  # Adjust this based on the actual dataset name
y_dataset_name = 'y_train'  # Adjust this based on the actual dataset name
x_dataset_name2 = 'x_test'  # Adjust this based on the actual dataset name
y_dataset_name2 = 'y_test'  # Adjust this based on the actual dataset name

# Load data
x_train = load_data(x_train_file, x_dataset_name)
x_train = tf.expand_dims(x_train, axis=-1)
y_train = load_data(y_train_file, y_dataset_name)
print(np.shape(x_train))
print(np.shape(y_train))
# Duplicate the binary labels to create two columns
#y_train = np.stack((y_train, 1 - y_train), axis=1)
x_test_file = '/Users/lucabernecker/Desktop/N128_local/steno_32_xxl_seg/x_test.h5'
y_test_file = '/Users/lucabernecker/Desktop/N128_local/steno_32_xxl_seg/y_test.h5'

# Load data
x_test, y_test = load_data(x_test_file,x_dataset_name2), load_data(y_test_file,y_dataset_name2)
x_test = tf.expand_dims(x_test, axis=-1)
num_classes = 1
input_shape = (32, 32, 32,1)

(1265, 32, 32, 32, 1)
(1265,)


In [6]:
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 25
num_epochs = 10  # For real training, use num_epochs=100. 10 is a test value
image_size = 100  # We'll resize input images to this size
patch_size = 10  # Size of the patches to be extract from the input images
num_patches = (image_size // patch_size) ** 3
projection_dim = 256
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 8
mlp_head_units = [
    2048,
    1024,
]  # Size of the dense layers of the final classifier
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=keras.activations.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [7]:
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.Resizing(image_size, image_size),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(factor=0.02),
        layers.RandomZoom(height_factor=0.2, width_factor=0.2),
    ],
    name="data_augmentation",
)
# Compute the mean and the variance of the training data for normalization.
data_augmentation.layers[0].adapt(x_train)

In [111]:
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super().__init__()
        self.patch_size = patch_size

    def call(self, images):
        input_shape = ops.shape(images)
        batch_size = input_shape[0]
        height = input_shape[1]
        width = input_shape[2]
        channels = input_shape[3]
        num_patches_h = height // self.patch_size
        num_patches_w = width // self.patch_size
        patches = keras.ops.image.extract_patches(images, size=self.patch_size)
        patches = ops.reshape(
            patches,
            (
                batch_size,
                num_patches_h * num_patches_w,
                self.patch_size * self.patch_size * channels,
            ),
        )
        return patches

    def get_config(self):
        config = super().get_config()
        config.update({"patch_size": self.patch_size})
        return config
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers



In [12]:
class RandomRotation3D(layers.Layer):
    def __init__(self, factor):
        super().__init__()
        self.factor = factor

    def call(self, inputs, training=True):
        if not training:
            return inputs

        # Generate random rotation angles for each axis
        angles = self.factor * tf.random.uniform([3], -1, 1)
        inputs = self.rotate_along_axis(inputs, angle=angles[0], axis=1)
        inputs = self.rotate_along_axis(inputs, angle=angles[1], axis=2)
        inputs = self.rotate_along_axis(inputs, angle=angles[2], axis=3)
        return inputs

    def rotate_along_axis(self, inputs, angle, axis):
        # Create the rotation matrix
        cos_angle = tf.cos(angle)
        sin_angle = tf.sin(angle)
        one = tf.ones_like(cos_angle)
        zero = tf.zeros_like(cos_angle)

        if axis == 1:
            rotation_matrix = tf.convert_to_tensor([[one, zero, zero],
                                                    [zero, cos_angle, -sin_angle],
                                                    [zero, sin_angle, cos_angle]])
        elif axis == 2:
            rotation_matrix = tf.convert_to_tensor([[cos_angle, zero, sin_angle],
                                                    [zero, one, zero],
                                                    [-sin_angle, zero, cos_angle]])
        else:
            rotation_matrix = tf.convert_to_tensor([[cos_angle, -sin_angle, zero],
                                                    [sin_angle, cos_angle, zero],
                                                    [zero, zero, one]])

        rotation_matrix = tf.reshape(rotation_matrix, [3, 3])
        shape = tf.shape(inputs)
        flat_inputs = tf.reshape(inputs, [-1, shape[1], shape[2], shape[3], shape[4]])
        rotated_inputs = tf.einsum('bij,bjklm->biklm', rotation_matrix, flat_inputs)
        return tf.reshape(rotated_inputs, shape)

class RandomRotation3D(layers.Layer):
    def __init__(self, factor):
        super().__init__()
        self.factor = factor

    def call(self, inputs, training=True):
        if not training:
            return inputs

        # Generate random rotation angles for each axis
        angles = self.factor * tf.random.uniform([3], -1, 1)
        inputs = self.rotate_along_axis(inputs, angle=angles[0], axis=1)
        inputs = self.rotate_along_axis(inputs, angle=angles[1], axis=2)
        inputs = self.rotate_along_axis(inputs, angle=angles[2], axis=3)
        return inputs

    def rotate_along_axis(self, inputs, angle, axis):
        # Create the rotation matrix
        cos_angle = tf.cos(angle)
        sin_angle = tf.sin(angle)
        one = tf.ones_like(cos_angle)
        zero = tf.zeros_like(cos_angle)

        if axis == 1:
            rotation_matrix = tf.convert_to_tensor([[one, zero, zero],
                                                    [zero, cos_angle, -sin_angle],
                                                    [zero, sin_angle, cos_angle]])
        elif axis == 2:
            rotation_matrix = tf.convert_to_tensor([[cos_angle, zero, sin_angle],
                                                    [zero, one, zero],
                                                    [-sin_angle, zero, cos_angle]])
        else:
            rotation_matrix = tf.convert_to_tensor([[cos_angle, -sin_angle, zero],
                                                    [sin_angle, cos_angle, zero],
                                                    [zero, zero, one]])

        rotation_matrix = tf.reshape(rotation_matrix, [3, 3])
        shape = tf.shape(inputs)
        flat_inputs = tf.reshape(inputs, [-1, shape[2], shape[3], shape[4]])
        rotated_inputs = tf.einsum('ij,bjklm->biklm', rotation_matrix, flat_inputs)
        return tf.reshape(rotated_inputs, shape)
class RandomZoom3D(layers.Layer):
    def __init__(self, height_factor, width_factor, depth_factor):
        super().__init__()
        self.height_factor = height_factor
        self.width_factor = width_factor
        self.depth_factor = depth_factor

    def call(self, inputs, training=True):
        if not training:
            return inputs

        # Apply zoom on each dimension
        zoom_factors = [1 + tf.random.uniform([], -self.depth_factor, self.depth_factor),
                        1 + tf.random.uniform([], -self.height_factor, self.height_factor),
                        1 + tf.random.uniform([], -self.width_factor, self.width_factor)]

        shape = tf.shape(inputs)
        zoomed = tf.image.resize(inputs, [tf.cast(shape[1] * zoom_factors[0], tf.int32),
                                          tf.cast(shape[2] * zoom_factors[1], tf.int32),
                                          tf.cast(shape[3] * zoom_factors[2], tf.int32)])

        return tf.image.resize_with_crop_or_pad(zoomed, shape[1], shape[2], shape[3])

# Assuming x_train has the shape (1265, 32, 32, 32, 1)
print()
# Assuming x_train has the shape (1265, 32, 32, 32)


# Define the image size
image_size = (32, 32, 32,1)

# Define the data augmentation pipeline for 3D images
data_augmentation = tf.keras.Sequential(
    [
        layers.Normalization(),
   #     layers.Resizing(image_size[0], image_size[1], image_size[2]),
   #     layers.RandomFlip("horizontal"),
   #     RandomRotation3D(factor=0.0),
   #     RandomZoom3D(height_factor=0, width_factor=0, depth_factor=0),
    ],
    name="data_augmentation",
)



In [29]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super().__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patches):
        batch_size = tf.shape(patches)[0]
        positions = tf.expand_dims(tf.range(start=0, limit=self.num_patches, delta=1), axis=0)
        positions = tf.tile(positions, [batch_size, 1])
        projected_patches = self.projection(patches)
        encoded_patches = projected_patches[:, :self.num_patches, :] + self.position_embedding(positions)

        # Slice to ensure consistent patch size
        encoded_patches = encoded_patches

        return encoded_patches

    def get_config(self):
        config = super().get_config()
        config.update({"num_patches": self.num_patches})
        return config

class Patches3D(layers.Layer):
    def __init__(self, patch_size, **kwargs):
        super().__init__(**kwargs)
        self.patch_size = patch_size

    def call(self, volumes):
        batch_size = tf.shape(volumes)[0]
        depth = volumes.shape[1]
        height = volumes.shape[2]
        width = volumes.shape[3]
        channels = volumes.shape[4]

        patches = tf.image.extract_patches(
            images=tf.reshape(volumes, [-1, height, width, channels]),
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding='VALID'
        )

        patch_depth = depth // self.patch_size
        patch_height = height // self.patch_size
        patch_width = width // self.patch_size

        patches = tf.reshape(patches, [batch_size, patch_depth, patch_height, patch_width, -1])
        patches = tf.transpose(patches, [0, 1, 2, 3, 4])
        patches = tf.reshape(patches, [batch_size, -1, self.patch_size * self.patch_size * channels])

  #      print(f"Patches3D: volumes shape={volumes.shape}, patches shape={patches.shape}")

        return patches

# Assuming x_train has the shape (1265, 32, 32, 32, 1)
x_train_expanded = tf.expand_dims(x_train, axis=-1)

# Now create the ViT classifier
def create_vit_classifier(input_shape=(32, 32, 32, 1), num_classes=1):
    patch_size = 4
    num_patches = (input_shape[0] // patch_size) * (input_shape[1] // patch_size) * (input_shape[2] // patch_size)
    projection_dim = 256
    transformer_layers = 8

    inputs = keras.Input(shape=input_shape)
    # Augment data.
    augmented = data_augmentation(inputs)
    # Create patches.
    patches = Patches3D(patch_size)(augmented)
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

  #  print(f"create_vit_classifier: input shape={input_shape}, patches shape={patches.shape}, encoded_patches shape={encoded_patches.shape}")

    for _ in range(transformer_layers):
        encoded_patches = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        attention_output = layers.MultiHeadAttention(num_heads=4, key_dim=projection_dim, dropout=0.1)(encoded_patches, encoded_patches)
        attention_output = layers.Dropout(0.1)(attention_output)
        encoded_patches = layers.Add()([encoded_patches, attention_output])

        feedforward_output = layers.Dense(projection_dim, activation=tf.nn.gelu)(encoded_patches)
        feedforward_output = layers.Dropout(0.1)(feedforward_output)
        encoded_patches = layers.Add()([encoded_patches, feedforward_output])

    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.1)(representation)
    outputs = layers.Dense(units=num_classes, activation='sigmoid')(representation)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

vit_classifier = create_vit_classifier(input_shape=(32, 32, 32, 1))
#vit_classifier.summary()
from tensorflow.keras.optimizers.legacy import Adam
vit_classifier.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='binary_crossentropy',  # Adjust loss function as needed
    metrics=[tf.keras.metrics.BinaryAccuracy()]  # Adjust metrics as needed
)

# Train the model
history = vit_classifier.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    batch_size=25,
    epochs=10,
    validation_split=0.2
)


Epoch 1/10
 1/51 [..............................] - ETA: 7:31 - loss: 0.7572 - binary_accuracy: 0.7600

KeyboardInterrupt: 

ValueError: The last dimension of the inputs to a Dense layer should be defined. Found None. Full input shape received: (10000, None)

TypeError: Exception encountered when calling layer "masked_patch_encoder_1" (type MaskedPatchEncoder).

in user code:

    File "/var/folders/l5/n49gt_b5117516rctl7_nsx40000gn/T/ipykernel_53352/814724563.py", line 16, in call  *
        mask, mask_indices = mask_patches(patches, self.mask_ratio)
    File "/var/folders/l5/n49gt_b5117516rctl7_nsx40000gn/T/ipykernel_53352/612595237.py", line 5, in mask_patches  *
        num_masked = int(mask_ratio * num_patches)

    TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'


Call arguments received by layer "masked_patch_encoder_1" (type MaskedPatchEncoder):
  • patches=tf.Tensor(shape=(None, None, 16), dtype=float32)

In [ ]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super().__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patches):
        batch_size = tf.shape(patches)[0]
        positions = tf.expand_dims(tf.range(start=0, limit=self.num_patches, delta=1), axis=0)
        positions = tf.tile(positions, [batch_size, 1])
        projected_patches = self.projection(patches)
        encoded_patches = projected_patches[:, :self.num_patches, :] + self.position_embedding(positions)

        # Slice to ensure consistent patch size
        encoded_patches = encoded_patches

        return encoded_patches

    def get_config(self):
        config = super().get_config()
        config.update({"num_patches": self.num_patches})
        return config

class Patches3D(layers.Layer):
    def __init__(self, patch_size, **kwargs):
        super().__init__(**kwargs)
        self.patch_size = patch_size

    def call(self, volumes):
        batch_size = tf.shape(volumes)[0]
        depth = volumes.shape[1]
        height = volumes.shape[2]
        width = volumes.shape[3]
        channels = volumes.shape[4]

        patches = tf.image.extract_patches(
            images=tf.reshape(volumes, [-1, height, width, channels]),
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding='VALID'
        )

        patch_depth = depth // self.patch_size
        patch_height = height // self.patch_size
        patch_width = width // self.patch_size

        patches = tf.reshape(patches, [batch_size, patch_depth, patch_height, patch_width, -1])
        patches = tf.transpose(patches, [0, 1, 2, 3, 4])
        patches = tf.reshape(patches, [batch_size, -1, self.patch_size * self.patch_size * channels])

  #      print(f"Patches3D: volumes shape={volumes.shape}, patches shape={patches.shape}")

        return patches

# Assuming x_train has the shape (1265, 32, 32, 32, 1)
x_train_expanded = tf.expand_dims(x_train, axis=-1)

# Now create the ViT classifier
def create_vit_classifier(input_shape=(32, 32, 32, 1), num_classes=1):
    patch_size = 4
    num_patches = (input_shape[0] // patch_size) * (input_shape[1] // patch_size) * (input_shape[2] // patch_size)
    projection_dim = 256
    transformer_layers = 8

    inputs = keras.Input(shape=input_shape)
    # Augment data.
    augmented = data_augmentation(inputs)
    # Create patches.
    patches = Patches3D(patch_size)(augmented)
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

  #  print(f"create_vit_classifier: input shape={input_shape}, patches shape={patches.shape}, encoded_patches shape={encoded_patches.shape}")

    for _ in range(transformer_layers):
        encoded_patches = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        attention_output = layers.MultiHeadAttention(num_heads=4, key_dim=projection_dim, dropout=0.1)(encoded_patches, encoded_patches)
        attention_output = layers.Dropout(0.1)(attention_output)
        encoded_patches = layers.Add()([encoded_patches, attention_output])

        feedforward_output = layers.Dense(projection_dim, activation=tf.nn.gelu)(encoded_patches)
        feedforward_output = layers.Dropout(0.1)(feedforward_output)
        encoded_patches = layers.Add()([encoded_patches, feedforward_output])

    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.1)(representation)
    outputs = layers.Dense(units=num_classes, activation='sigmoid')(representation)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

vit_classifier = create_vit_classifier(input_shape=(32, 32, 32, 1))
#vit_classifier.summary()
from tensorflow.keras.optimizers.legacy import Adam
vit_classifier.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='binary_crossentropy',  # Adjust loss function as needed
    metrics=[tf.keras.metrics.BinaryAccuracy()]  # Adjust metrics as needed
)

# Train the model
history = vit_classifier.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    batch_size=25,
    epochs=10,
    validation_split=0.2
)